In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [3]:
df = pd.read_pickle('../savedFiles/cleaned_df.pkl')

In [4]:
df.reset_index()

,index,title,text,link,author,label,clean_title,clean_text
0,0,"A Collapse That Wiped Out 5 Years of Growth, W...","“Not only have we plateaued, but we may be los...",https://www.nytimes.com/2020/07/30/business/ec...,[Ben Casselman],0,collapse wiped years growth bounce sight,plateaued may losing ground said diane swonk ...
1,1,"Grilled by Lawmakers, Big Tech Turns Up the Ga...","“I don’t know the specifics of that situation,...",https://www.nytimes.com/2020/07/30/technology/...,[Kevin Roose],0,grilled lawmakers big tech turns gaslight,know specifics situation mr bezos replied ano...
2,2,How Pimco’s Cayman-Based Hedge Fund Can Profit...,WASHINGTON — Pacific Investment Management Com...,https://www.nytimes.com/2020/07/30/business/ec...,[Jeanna Smialek],0,pimco cayman based hedge fund profit fed rescue,washington pacific investment management comp...
3,3,Big Tech Earnings Surge as Economy Slumps,"Live Updated July 30, 2020, 8:22 p.m. ET July ...",https://www.nytimes.com/live/2020/07/30/busine...,"[Mike Isaac, Karen Weise, Daisuke Wakabayashi,...",0,big tech earnings surge economy slumps,live updated july p et july p et big tech ear...
4,4,‘This Is a New Phase’: Europe Shifts Tactics t...,LONDON — European Union leaders are pursuing a...,https://www.nytimes.com/2020/07/30/technology/...,[Adam Satariano],0,new phase europe shifts tactics limit tech power,london european union leaders pursuing new la...
...,...,...,...,...,...,...,...,...
799,799,BREAKING! This Revolutionary Cancer-Killing Tr...,This is the future of medicine. Novartis Pharm...,https://www.usasupreme.com/breaking-revolution...,"[Alex D., Alex D Is A Conservative Journalist,...",1,breaking revolutionary cancer killing treatme...,future medicine novartis pharmaceuticals comp...
800,800,This Magic Mushroom Compound Will Help Treat D...,Science is definitely amazing!\n\nNature is ab...,https://www.usasupreme.com/magic-mushroom-comp...,"[Natalie D., Natalie D. Is An American Conserv...",1,magic mushroom compound help treat depression,science definitely amazing nature abundant tr...
801,801,This Magic Mushroom Compound Will Help Treat D...,Science is definitely amazing!\n\nNature is ab...,https://www.usasupreme.com/magic-mushroom-comp...,"[Natalie D., Natalie D. Is An American Conserv...",1,magic mushroom compound help treat depression,science definitely amazing nature abundant tr...
802,802,Trump Appoint Hollywood Legend To Be The Head ...,Presiden Trump made serious changes in the Whi...,https://www.usasupreme.com/trump-appoint-holly...,"[Alex D., Alex D Is A Conservative Journalist,...",1,trump appoint hollywood legend head america f...,presiden trump made serious changes white hou...


In [5]:
# Preparing the target and predictors for modeling
# Keep the title and body text separated for different models

X_body_text = df['clean_text'].values
X_title_text = df['clean_title'].values
y = df['label'].values

# create tf idf vector

In [6]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df= 0.85, min_df= 0.01)

In [7]:
X_body_tfidf = tfidf.fit_transform(X_body_text)
X_title_tfidf = tfidf.fit_transform (X_title_text)

In [8]:
indices = df.index.values

# 80% of articles for training, 20% for testing from X and y

In [9]:
X_body_tfidf_train, X_body_tfidf_test, y_body_train, y_body_test, \
indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)

In [10]:
df.loc[indices_body_train].groupby('label').agg('count')  #training data states 

,title,text,link,author,clean_title,clean_text
label,,,,,,
0,473,473,473,473,473,473
1,170,170,170,170,170,170


In [11]:
df.loc[indices_body_test].groupby('label').agg('count')   #test data states

,title,text,link,author,clean_title,clean_text
label,,,,,,
0,117,117,117,117,117,117
1,44,44,44,44,44,44


# Import naive bayes classifier

In [12]:
from sklearn.naive_bayes import MultinomialNB

In [13]:
nb_body = MultinomialNB()        #initialize nb classifier

fit training data into classifier

In [14]:
nb_body.fit(X_body_tfidf_train, y_body_train)

MultinomialNB()

prediction 

In [15]:
y_body_train_pred = nb_body.predict(X_body_tfidf_train)

In [16]:
print('Naive Bayes In Sample F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(y_body_train, y_body_train_pred, average='macro')*100 ))
print ('Accuracy score {:.4}%'.format(accuracy_score(y_body_train, y_body_train_pred)*100))

Naive Bayes In Sample F1 and Accuracy Scores:
F1 score 84.7%
Accuracy score 89.74%


In [38]:
s = {'title':['west bengal', 'mumbai'],
     'text':['West Bengal Chief Minister Mamata Banerjee on Tuesday announced that lockdown across state for two days every week has been extended till August 31. There will be no lockdown in the state on the occasion of Eid on August 1, she added. The two designated lockdown days for every week would be announced in advance, the government had announced earlier.',
                       'Mumbai reported 700 coronavirus cases in a day on Tuesday, the lowest in three months. The good news: Only 700 cases today in Mumbai and that too with highest testing till date in Mumbai in a single day (8,776)...A major relief after 3 months, Maharashtra minister Aaditya Thackeray tweeted. Dont let your mask down! Only get numbers down! he added.']}

In [39]:
df = pd.DataFrame(s)

In [40]:
df

,title,text
0,west bengal,West Bengal Chief Minister Mamata Banerjee on ...
1,mumbai,Mumbai reported 700 coronavirus cases in a day...


In [41]:
%run cleaning_helper.py

In [42]:
new_df = df.copy()

In [43]:
new_df = clean_column(new_df, 'title', 'clean_title')
new_df = clean_column(new_df, 'text', 'clean_text')
new_df.head()

,title,text,clean_title,clean_text
0,west bengal,West Bengal Chief Minister Mamata Banerjee on ...,west bengal,west bengal chief minister mamata banerjee tu...
1,mumbai,Mumbai reported 700 coronavirus cases in a day...,mumbai,mumbai reported coronavirus cases day tuesday...


In [44]:
text_values = new_df['clean_text'].values
x_body_tfidf = tfidf.fit_transform(text_values)

In [45]:
y_body_train_pred = nb_body.predict(x_body_tfidf)

ValueError: dimension mismatch